# Bing Maps API

In this part of the lab, you will join the CityBikes data from the previous part of the lab with additional data that you will query from the Bing Maps API.

First, you will need to register for a Bing Maps Key. Follow the instructions [here](https://docs.microsoft.com/en-us/bingmaps/getting-started/bing-maps-dev-center-help/getting-a-bing-maps-key). You should be able to sign in with your Cal Poly account. When you get to Step 4, select:

- Key type: Basic
- Application type: Dev/Test

You will be able to make 125000 free requests with the resulting API key. This should be more than enough to complete this assignment.

We will be working with the [REST services in the Bing Maps API](https://docs.microsoft.com/en-us/bingmaps/rest-services/). Click on the link for a complete documentation of the features.

## Question 1

Read in the `DataFrame` of bike stations in the United States from Part A of this lab. Restrict to the stations in the "Bay Wheels" network (with network ID "ford-gobike").

How many of these stations are in the city/county of San Francisco? 

(_Hint:_ Use the [Locations API](https://docs.microsoft.com/en-us/bingmaps/rest-services/locations/) to get the address associated with each latitude and longitude coordinate.)

In [142]:
import pandas as pd
import requests
from google.colab import files
import json
import pandas as pd
from pandas.io.json import json_normalize
uploaded = files.upload()
df_us_bikes = pd.read_csv("us_bike_stations.csv")

In [143]:
df_bay = df_us_bikes[df_us_bikes["network.name"] =="Bay Wheels"]
df_bay

,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,extra.address,extra.last_updated,extra.renting,extra.returning,extra.uid,network.name,extra.ebikes,extra.has_ebikes,extra.bike_uids,extra.number,extra.slots
6231,10.0,5.0,d0e8f4f1834b7b33a3faf8882f567ab8,37.849735,-122.270582,Harmon St at Adeline St,2022-02-23T03:54:22.093000Z,NaN,1.645575e+09,1.0,1.0,340,Bay Wheels,0.0,True,NaN,NaN,NaN
6232,5.0,6.0,983514094dd808b1604da2dcfc2d09af,37.336188,-121.889277,Fountain Alley at S 2nd St,2022-02-23T03:54:22.096000Z,NaN,1.645533e+09,1.0,1.0,341,Bay Wheels,0.0,True,NaN,NaN,NaN
6233,3.0,20.0,da17603652106fda93da4e255a5b0a22,37.322125,-121.881090,Oak St at 1st St,2022-02-23T03:54:22.100000Z,NaN,1.645579e+09,1.0,1.0,298,Bay Wheels,0.0,True,NaN,NaN,NaN
6234,2.0,13.0,7a21c92b3b4cd2f7759107b4fdebf869,37.323678,-121.874119,Bestor Art Park,2022-02-23T03:54:22.101000Z,NaN,1.645532e+09,1.0,1.0,299,Bay Wheels,0.0,True,NaN,NaN,NaN
6235,22.0,5.0,ce34d38fb230a23c1ced12d1e16df294,37.325998,-121.877120,5th St at Virginia St,2022-02-23T03:54:22.106000Z,NaN,1.645574e+09,1.0,1.0,296,Bay Wheels,0.0,True,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6708,18.0,5.0,df9c9c79952699cc041118a331ea9cc2,37.724893,-122.443238,Balboa Park (San Jose Ave at Sgt. John V. Youn...,2022-02-23T03:54:23.195000Z,NaN,1.645575e+09,1.0,1.0,501,Bay Wheels,4.0,True,NaN,NaN,NaN
6709,17.0,10.0,5dc0d98ea835b4c9743d4e2cdd515dde,37.726266,-122.479462,SFSU University Park North - West,2022-02-23T03:54:23.401000Z,NaN,1.645584e+09,1.0,1.0,566,Bay Wheels,0.0,True,NaN,NaN,NaN
6710,4.0,11.0,97871d03b345e3aa1de1dbed3640b700,37.725603,-122.476561,SFSU University Park North - East,2022-02-23T03:54:23.402000Z,NaN,1.645581e+09,1.0,1.0,567,Bay Wheels,7.0,True,NaN,NaN,NaN
6711,4.0,15.0,e427fa5d07c952f21297dbd9c7693407,37.723426,-122.455102,Brighton Ave at Ocean Ave,2022-02-23T03:54:23.399000Z,NaN,1.645535e+09,1.0,1.0,565,Bay Wheels,5.0,True,NaN,NaN,NaN


In [144]:
response = requests.get("http://dev.virtualearth.net/REST/v1/Locations/37.802747,-122.405861?&key=Auc0oTBQfFANjo26Qvhu4-ShGKMPCONB0X0y28vKq-VBu66awkpJIDJwW5Ko3DES")
response.text[:1000]


'{"authenticationResultCode":"ValidCredentials","brandLogoUri":"http:\\/\\/dev.virtualearth.net\\/Branding\\/logo_powered_by.png","copyright":"Copyright © 2022 Microsoft and its suppliers. All rights reserved. This API cannot be accessed and the content and any results may not be used, reproduced or transmitted in any manner without express written permission from Microsoft Corporation.","resourceSets":[{"estimatedTotal":1,"resources":[{"__type":"Location:http:\\/\\/schemas.microsoft.com\\/search\\/local\\/ws\\/rest\\/v1","bbox":[37.79888428242932,-122.41237897714247,37.806609717570673,-122.39934302285754],"name":"54 Telegraph Hill Blvd, San Francisco, CA 94133, United States","point":{"type":"Point","coordinates":[37.802747,-122.405861]},"address":{"addressLine":"54 Telegraph Hill Blvd","adminDistrict":"CA","adminDistrict2":"San Francisco Co.","countryRegion":"United States","formattedAddress":"54 Telegraph Hill Blvd, San Francisco, CA 94133, United States","intersection":{"baseStreet

In [145]:
df_sf = json_normalize(response.json(), ["resourceSets", "resources"])
x= df_sf["address.adminDistrict2"] == "San Francisco Co."
x[0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


True

In [ ]:
import time


df_sf = []
for x in range(6231,6713):
    la = df_us_bikes["latitude"].iloc[x]
    lo = df_us_bikes["longitude"].iloc[x]
    latlong = la, lo
    response = requests.get("http://dev.virtualearth.net/REST/v1/Locations/%la,%la?&key=Auc0oTBQfFANjo26Qvhu4-ShGKMPCONB0X0y28vKq-VBu66awkpJIDJwW5Ko3DES" % latlong)
    sf = json_normalize(response.json(), ["resourceSets", "resources"])
    if (sf["address.adminDistrict2"] == "San Francisco Co.")[0]:
      df_sf.append(sf)
    time.sleep(0.499)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
df_sf_sf = pd.concat(df_sf)
len(df_sf_sf)

## Question 2

You want to go to Coit Tower. To save money, you decide to ride a "Bay Wheels" bike to the closest station and hail a cab from there to Coit Tower. What station should you bike to so that you are as close to Coit Tower as possible (as measured by driving distance)? Does your answer agree with the one that you obtained in Part A of this lab? If not, why does it differ?

_Hints:_ 
- You should restrict your attention to bike stations that are in San Francisco, which you determined in Question 1. 
- Use the [Routes API](https://docs.microsoft.com/en-us/bingmaps/rest-services/routes/calculate-a-distance-matrix) to calculate a distance matrix between Coit Tower and the bike stations. 
- You can do this with just one call to the API. Because there are too many stations, it is impossible to specify all the locations in the URL. Instead, you should make a POST request (`requests.post`), passing in the parameters as a JSON object through the `json=` parameter of `requests.post`. Read the API documentation carefully to learn how to use the POST API.

In [ ]:
import numpy as np
df_sf_sf['index'] = np.arange(len(df_sf_sf))
df_sf_sf = df_sf_sf.set_index('index')
df_sf_sf.head()

In [ ]:
arr = []
for x in range(0, len(df_sf_sf)):
  temp_dict = {"latitude": df_sf_sf["point.coordinates"][x][0],
              "longitude": df_sf_sf["point.coordinates"][x][1]}
  arr.append(temp_dict)

In [ ]:
json_object = {
    "origins": arr,
    "destinations": [{
        "latitude": 37.8024,
        "longitude": -122.4058
    }],
    "travelMode": "driving"
}

In [ ]:
response = requests.post("https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?key=Auc0oTBQfFANjo26Qvhu4-ShGKMPCONB0X0y28vKq-VBu66awkpJIDJwW5Ko3DES", json= json_object)


In [ ]:
info= json_normalize(response.json(), ["resourceSets", "resources", "results"])
df_sf_sf["name"][info["travelDistance"].idxmin()]

This distance is different from the one in part A because we are now actually following the roads instead of measuring distance as if there were no obstacles.

## Submission Instructions

- Copy this notebook to your own Drive, if you have not already.
- Restart this notebook and run the cells from beginning to end. 
  - Go to Runtime > Restart and Run All.
- Rename this notebook by clicking on "DATA 301 Lab 6B - YOUR NAMES HERE" at the very top of this page. Replace "YOUR NAMES HERE" with the first and last names of you (and your partners, for Phase 2).
- Get the link to your notebook:
  - Click on "Share" at the top-right. 
  - Change the settings to "Anyone with the link can view". 
  - Copy the sharing link into Canvas.